In [1]:
# Base on wandb documentation: Scikit-learn integration
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.exceptions import ConvergenceWarning
import warnings
import pickle
import wandb

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, roc_curve, mean_squared_error

warnings.filterwarnings("ignore", category=ConvergenceWarning)

# import weave

In [2]:
# Failed to detect the name of this notebook...
os.environ['WANDB_NOTEBOOK_NAME'] = '04_opiod_wandb_tree.ipynb'

In [3]:
wandb.login()

wandb: Currently logged in as: idiazl (dev_ml_ops). Use `wandb login --relogin` to force relogin


True

In [4]:
data = 'CaseStudy_training_data.xlsx'
df = pd.read_excel(data, sheet_name='Model_data')

3. Data cleaning

In [5]:
df_cleaned = df.dropna()
df_cleaned = df_cleaned.drop_duplicates()
df_cleaned = df_cleaned.drop(['ID'], axis=1)
df_cleaned = df_cleaned.rename(columns={'rx ds': 'rx_ds'})

4. Feature Engineering

In [6]:
# Perform percentile-based bucketing for 'rx_ds'
df_features = df_cleaned.copy()
df_features['rx_ds_bucket'] = pd.qcut(
    df_cleaned['rx_ds'], 
    q=4, 
    labels=['Q1', 'Q2', 'Q3', 'Q4']
    )

In [7]:
# Create a new feature that is the sum of all the binary features
binary_cols = [col for col in df_features.columns if col not in ['OD', 'rx_ds', 'rx_ds_bucket']]
df_features['binary_sum'] = df_features[binary_cols].sum(axis=1)

# Create a new feature that is the ratio of 'rx_ds' to the sum of binary features
df_features['rx_ds_to_binary_sum'] = df_features['rx_ds'] / df_features['binary_sum']

In [8]:
# Perform one-hot encoding for 'rx_ds_bucket'
df_one_hot = pd.get_dummies(df_features['rx_ds_bucket'], prefix='rx_ds_bucket')
df_features = pd.concat([df_features, df_one_hot], axis=1)
df_features.drop(['rx_ds_bucket'], axis=1, inplace=True)

## Runs for model training

### 1. Classification - Decision Tree

In [9]:
from sklearn.model_selection import train_test_split

df_tree = df_features.copy()

X = df_tree.drop(['OD', 'rx_ds'], axis=1)
y = df_tree['OD']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Defining a function to calculate the metrics for the classifier
def calculate_metrics(y_test, y_pred, model, X_test):
    metrics = {}
    metrics["accuracy"] = accuracy_score(y_test, y_pred)
    metrics["precision"] = precision_score(y_test, y_pred)
    metrics["recall"] = recall_score(y_test, y_pred)
    metrics["f1"] = f1_score(y_test, y_pred)
    metrics["roc_auc"] = roc_auc_score(y_test, model.predict_proba(X_test)[:,1])
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    TN, FP, FN, TP = conf_matrix.ravel()
    
    metrics["ppv"] = TP / (TP + FP)
    metrics["npv"] = TN / (TN + FN)
    metrics["specificity"] = TN / (TN + FP)
    
    return metrics

In [21]:
# New wandb project and run
run = wandb.init(project='wandb-sklearn-tree', name="classifier_decision_tree")

tree_params = {
    'penalty': 'l1',
    'solver': 'liblinear',
    'C': 5,
    'class_weight': 'balanced',
    'max_iter': 5000,
    'random_state': 42
}

wandb.config = tree_params

In [22]:
# Initializing the model, Fitting and predicting
tree_model = LogisticRegression(**tree_params)
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)

### Logging metrics

In [23]:
# Logging the performance metrics
y_pred = tree_model.predict(X_test)
tree_metrics = calculate_metrics(y_test, y_pred, tree_model, X_test)

wandb.log(tree_metrics)

#### Logging artifacts

In [24]:
# Save the model
os.makedirs('models', exist_ok=True)
with open("models/tree_metrics.pkl", "wb") as f:
    pickle.dump(log_model, f)

# Log the model as a versioned file
artifact = wandb.Artifact("tree_mode", type="model")
artifact.add_file("models/tree_model.pkl")
wandb.log_artifact(artifact)

<Artifact log_mode>

In [17]:
# Save the data
os.makedirs('data', exist_ok=True)
datasets = {"trainig": X, "validation": y}

for name, df in datasets.items():
    df.to_csv(f'data/{name}.csv', index=False)

# Log the `data` as an artifact
artifact = wandb.Artifact('train_val_sets', type='dataset', metadata={"Source": "CaseStudy_training_data.xlsx"})
artifact.add_dir('data')
wandb.log_artifact(artifact)

wandb: Adding directory to artifact (./data)... Done. 0.0s


<Artifact train_val_sets>

In [25]:
wandb.finish()

accuracy,▁
f1,▁
npv,▁
ppv,▁
precision,▁
recall,▁
roc_auc,▁
specificity,▁
accuracy,0.715
f1,0.6069
npv,0.86842
